In [1]:
from tdmclient import ClientAsync
import vision.ComputerVision
import time
import cv2
import motion_functions
import filtering
import numpy as np

dt = 0.05 # [s] Time delta between minimum image acquisition

vis = vision.ComputerVision.Vision()

client = ClientAsync()
node = await client.wait_for_node()
await node.lock()

last_image_time = time.time()
dt = 0.05
path = []

state = 0 # FSM state
goal_reached = False
kidnapped = 0
obstacle_detected = 0

while True:
    while state == 0: # Goal, robot and path aquisition state
        while time.time()-last_image_time < dt: # Aquire a new image every dt seconds
            continue
        vis.show()
        vis.update()
        last_image_time = time.time()
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        rotation_done = 1 # Boolean to check if the robot is currently rotating (1 means no)
        target_node = 0 # Initial node
        
        path = vis.shortest_path
        # Check if a path was found
        if len(path)>=1:
            state = 1
            rob_pos = [vis.robot.x, vis.robot.y, -vis.robot.angle]
            filtering.f.x = np.array([vis.robot.x/vis.scale, vis.robot.y/vis.scale, -vis.robot.angle])

    while state == 1: # Drive towards goal state
        while time.time()-last_image_time < dt: # Aquire a new image every dt seconds
            continue
        vis.show()
        vis.update()
        
        await client.sleep(0.01)
        await node.wait_for_variables()
        
        # Get proximity sensors' values
        kidnapped, obstacle_detected = motion_functions.get_sensors(node)
        
        # Checking for unforseen obstacle
        if obstacle_detected:
            await motion_functions.stop(node)
            state = 4
            break
        
        # Checking for kidnapping
        if kidnapped:
            state = 3 # Kidnapped state
            vis.shortest_path = [] # Deleting the previous path to make sure a new one is computed
            await motion_functions.stop(node)
            break
        
        # Setting up and running the Kalman Filter
        speed_left = node.v.motor.left.speed
        speed_right = node.v.motor.right.speed
        rob_pos = filtering.run_filter(speed_right, speed_left, rob_pos[2], vis)
        #rob_pos = [vis.robot.x, vis.robot.y, -vis.robot.angle]
        
        # Checking if we arrived at the next node
        arrived_node = motion_functions.close_coordinates(rob_pos[0], rob_pos[1], path[target_node][0], path[target_node][1])
        if (arrived_node):
            if (target_node) < len(path)-1:
                target_node += 1
            else:
                await motion_functions.stop(node)
                goal_reached = True
                break 
        
        # Rotating and driving towards the next node
        if rotation_done:
            angle = motion_functions.compute_movement([rob_pos[0], rob_pos[1]], [path[target_node][0], path[target_node][1]])
            if abs(angle - rob_pos[2]) > 0.2:
                rotation_done = 0
                await motion_functions.rotate(angle-rob_pos[2], node)
            
        if  angle + 0.2 > rob_pos[2] and angle - 0.2 < rob_pos[2]:
            rotation_done = 1
            await motion_functions.drive(node)
        
            
        last_image_time = time.time()
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    while state == 3: # State in case of kidnapping
        while time.time()-last_image_time < dt: # Aquire a new image every dt seconds
            continue
        vis.show()
        vis.update()
        
        # Checking if the robot is still kidnapped
        await client.sleep(0.01)
        await node.wait_for_variables()
        kidnapped, obstacle_detected = motion_functions.get_sensors(node)
        
        if not kidnapped:
            state = 0
        
        last_image_time = time.time()
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
    while state == 4: # Local avoidance state
        vis.shortest_path = [] # Deleting the previous path to make sure a new one is computed
        
        while obstacle_detected:
            while time.time()-last_image_time < dt: # Aquire a new image every dt seconds
                continue
            vis.show()
            vis.update()
            
            # Checking if we still detect obstacles
            await client.sleep(0.01)
            await node.wait_for_variables()
            kidnapped, obstacle_detected = motion_functions.get_sensors(node)
            # Drive backwards as long as an obstacle is detected
            await motion_functions.drive_back(node)
            
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        
        # Stop and return to state 0 to compute a new path
        await motion_functions.stop(node)
        state = 0
    
    if goal_reached:
        break

cv2.destroyAllWindows()
del vis
await node.unlock()

2023-12-07 01:17:00.928 python[43148:1202361] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
100%|██████████| 1/1 [00:00<00:00, 1254.28it/s]


Det P 4.442978078651077e-07


error: OpenCV(4.8.1) :-1: error: (-5:Bad argument) in function 'circle'
> Overload resolution failed:
>  - Argument 'radius' is required to be an integer
>  - Argument 'radius' is required to be an integer


In [ ]:
await motion_functions.stop(node)
await node.unlock()